## Task 1
Here I'm generating the encodings for `k=2` for the graph generated by $\mathbb{Z}^1$

The problem can be represented using only the variables `X(i, c)` which means node `i` is assigned the color `c`.
We have the follow constraints:
* Every node has exactly one color
\begin{align*}
    \bigwedge_i \bigvee_c X(i, c) \land \bigwedge_i \bigwedge_{c_1 \neq c_2} \neg X(i, c_1) \lor \neg X(i, c_2)
\end{align*}
* If nodes $i_1$, $i_2$ both have color $c$, they're atleast $c$ distance apart.
\begin{align*}
    \bigwedge_c \bigwedge_{u, v, d(u, v) \leq c} \neg X(u, c) \lor \neg X(v, c)
\end{align*}

In [2]:
from pysat.formula import CNF
from itertools import combinations

We need to denote each $X(i, c)$ as an integer (as cnf files represent boolean
variables as integers). This can be done using
\begin{align*}
    X(i, c) \equiv i * k + c + 1
\end{align*}
where $k$ is the number of colors we have


In [50]:
def var(i, c, k):
    return i * k + c

def get_cnf(n: int, k: int) -> CNF:
    """
    n: number of nodes in graph
    k: number of colors
    """

    cnf = CNF()
    for i in range(n):
        cnf.append([var(i, c, k) for c in range(1, k + 1)])

    for i in range(n):
        for c0, c2 in combinations(range(1, k + 1), 2):
            cnf.append([-var(i, c0, k), -var(i, c2, k)])
    

    for c in range(0, k + 1):
        for i in range(n):
            for j in range(i, min(i + c + 1, n)):
                if i == j: continue
                cnf.append([-var(i, c, k), -var(j, c, k)])
    
    return cnf

In [51]:
n = 60
k = 3
cnf = get_cnf(60, 3)
cnf.to_file("Z1.cnf")

## Is it working?

You can run this by changing values of `n` and `k`. The code below can even be run from a seperate file as it just operates on `Z1.cnf` file produced by the encoding we made above. For now, I've hardcoded this to use `z3` and extract the results. And show the colors assigned to each real number

* n: Number of points we're considering to find a coloring. We'll choose the points [0, ..., n - 1] (position doesn't really matter).
* k: Number of colors we're taking

In [60]:
def get_results(n, k, **kwargs):

    file = "Z1.cnf"
    if "file" not in kwargs:
        cnf = get_cnf(n, k)
        cnf.to_file("Z1.cnf")
    else:
        file = kwargs["file"]

    import subprocess

    def unvar(var, k):
        var = var - 1
        return (var // k, var % k + 1)

    def get_colors(arr, k):
        colors = []
        for i in arr:
            if i <= 0: continue
            colors.append(unvar(i, k)[1])
        return colors

    result = subprocess.run(["z3", "-dimacs", "Z1.cnf"], capture_output=True, text=True)

    if (result.stdout.splitlines()[0] == "s UNSATISFIABLE"):
        print("UNSATISFIABLE")

    else:
        result = result.stdout.splitlines()[1]

        result = list(map(int, result.split()[1:-1]))
        print(f"Colors: {get_colors(result, k)}")

In [63]:
get_results(100, 3)

Colors: [3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1, 3, 1, 2, 1]
